In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50

print("Import Keras")

Using TensorFlow backend.


Import Keras


In [3]:
### Specify Training Parameters
batchSize = 64               #-- Training Batch Size
num_classes = 13             #-- Number of classes
num_epochs = 25              #-- Number of epochs for training   
#learningRate= 0.001          #-- Learning rate for the network
#lr_weight_decay = 0.95       #-- Learning weight decay. Reduce the learn rate by 0.95 after epoch

In [4]:
### Load base model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

80142336/80134624 [==============================] - 3s 0us/step


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 4s 0us/step


In [5]:
x1 = vgg16_model.output
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(512, activation='relu')(x1) # dense layer 1
x1 = Dropout(0.35)(x1)
x1 = Dense(128, activation='relu')(x1) # dense layer 2
x1 = Dropout(0.35)(x1)
pred1 = Dense(num_classes, activation='softmax')(x1) # final layer with softmax activation

model1 = Model(inputs=vgg16_model.input, outputs=pred1)
for layer in model1.layers[:20]:
    layer.trainable=False
for layer in model1.layers[20:]:
    layer.trainable=True
    
model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [6]:
x2 = vgg19_model.output
x2 = GlobalAveragePooling2D()(x2)
x2 = Dense(512, activation='relu')(x2) # dense layer 1
x2 = Dropout(0.35)(x2)
x2 = Dense(128, activation='relu')(x2) # dense layer 2
x2 = Dropout(0.35)(x2)
pred2 = Dense(num_classes, activation='softmax')(x2) # final layer with softmax activation

model2 = Model(inputs=vgg19_model.input, outputs=pred2)
for layer in model2.layers[:22]:
    layer.trainable=False
for layer in model2.layers[22:]:
    layer.trainable=True
    
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [7]:
x3 = resnet_model.output
x3 = GlobalAveragePooling2D()(x3)
x3 = Dense(2048, activation='relu')(x3) # dense layer 1
x3 = Dropout(0.35)(x3)
x3 = Dense(512, activation='relu')(x3)  # dense layer 2
x3 = Dropout(0.35)(x3)
x3 = Dense(128, activation='relu')(x3)  # dense layer 3
x3 = Dropout(0.35)(x3)
pred3 = Dense(num_classes, activation='softmax')(x3) # final layer with softmax activation

model3 = Model(inputs=resnet_model.input, outputs=pred3)
for layer in model3.layers[:175]:
    layer.trainable=False
for layer in model3.layers[175:]:
    layer.trainable=True
    
model3.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [8]:
"""x4 = base_model.output
x4 = GlobalAveragePooling2D()(x4)
x4 = Dense(512, activation='relu')(x4) # dense layer 1
x4 = Dropout(0.39)(x4)
x4 = Dense(128, activation='relu')(x4) # dense layer 2
x4 = Dropout(0.39)(x4)
pred4 = Dense(num_classes, activation='softmax')(x4) # final layer with softmax activation

model4 = Model(inputs=base_model.input, outputs=pred4)
for layer in model4.layers[:20]:
    layer.trainable=False
for layer in model4.layers[20:]:
    layer.trainable=True
    
model4.summary()"""

"x4 = base_model.output\nx4 = GlobalAveragePooling2D()(x4)\nx4 = Dense(512, activation='relu')(x4) # dense layer 1\nx4 = Dropout(0.39)(x4)\nx4 = Dense(128, activation='relu')(x4) # dense layer 2\nx4 = Dropout(0.39)(x4)\npred4 = Dense(num_classes, activation='softmax')(x4) # final layer with softmax activation\n\nmodel4 = Model(inputs=base_model.input, outputs=pred4)\nfor layer in model4.layers[:20]:\n    layer.trainable=False\nfor layer in model4.layers[20:]:\n    layer.trainable=True\n    \nmodel4.summary()"

In [9]:
"""x5 = base_model.output
x5 = GlobalAveragePooling2D()(x5)
x5 = Dense(512, activation='relu')(x5) # dense layer 1
x5 = Dropout(0.42)(x5)
x5 = Dense(128, activation='relu')(x5) # dense layer 2
x5 = Dropout(0.42)(x5)
pred5 = Dense(num_classes, activation='softmax')(x5) # final layer with softmax activation

model5 = Model(inputs=base_model.input, outputs=pred5)
for layer in model5.layers[:20]:
    layer.trainable=False
for layer in model5.layers[20:]:
    layer.trainable=True
    
model5.summary()"""

"x5 = base_model.output\nx5 = GlobalAveragePooling2D()(x5)\nx5 = Dense(512, activation='relu')(x5) # dense layer 1\nx5 = Dropout(0.42)(x5)\nx5 = Dense(128, activation='relu')(x5) # dense layer 2\nx5 = Dropout(0.42)(x5)\npred5 = Dense(num_classes, activation='softmax')(x5) # final layer with softmax activation\n\nmodel5 = Model(inputs=base_model.input, outputs=pred5)\nfor layer in model5.layers[:20]:\n    layer.trainable=False\nfor layer in model5.layers[20:]:\n    layer.trainable=True\n    \nmodel5.summary()"

In [10]:
from keras.applications.imagenet_utils import preprocess_input
train_datagen1 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies

train_generator1 = train_datagen1.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder
                                                     target_size=(256, 256),
                                                     batch_size=batchSize,
                                                     class_mode='categorical',
                                                     shuffle=True)

train_datagen2 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies

train_generator2 = train_datagen2.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder
                                                     target_size=(256, 256),
                                                     batch_size=batchSize,
                                                     class_mode='categorical',
                                                     shuffle=True)

train_datagen3 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies

train_generator3 = train_datagen3.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder
                                                     target_size=(256, 256),
                                                     batch_size=batchSize,
                                                     class_mode='categorical',
                                                     shuffle=True)

"""train_datagen4 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies

train_generator4 = train_datagen4.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder
                                                     target_size=(256, 256),
                                                     batch_size=batchSize,
                                                     class_mode='categorical',
                                                     shuffle=True)

train_datagen5 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies

train_generator5 = train_datagen5.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder
                                                     target_size=(256, 256),
                                                     batch_size=batchSize,
                                                     class_mode='categorical',
                                                     shuffle=True)"""

Found 2819 images belonging to 13 classes.
Found 2819 images belonging to 13 classes.
Found 2819 images belonging to 13 classes.


"train_datagen4 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies\n\ntrain_generator4 = train_datagen4.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder\n                                                     target_size=(256, 256),\n                                                     batch_size=batchSize,\n                                                     class_mode='categorical',\n                                                     shuffle=True)\n\ntrain_datagen5 = ImageDataGenerator(preprocessing_function=preprocess_input, horizontal_flip=True) #included in our dependencies\n\ntrain_generator5 = train_datagen5.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # specify the path to the main data folder\n                                                     target_size=(256, 256),\n                                               

In [11]:
label_map = train_generator1.class_indices
label_map = dict((v,k) for k,v in label_map.items())
for i in range(len(label_map)):
    print("class" + " {:>2}".format(str(i+1)) + ": " + label_map[i])

class  1: bedroom
class  2: coast
class  3: forest
class  4: highway
class  5: insidecity
class  6: kitchen
class  7: livingroom
class  8: mountain
class  9: office
class 10: opencountry
class 11: street
class 12: suburb
class 13: tallbuilding


In [12]:
model1.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model4.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model5.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train1 = train_generator1.n // train_generator1.batch_size
step_size_train2 = train_generator2.n // train_generator2.batch_size
step_size_train3 = train_generator3.n // train_generator3.batch_size
#step_size_train4 = train_generator4.n // train_generator4.batch_size
#step_size_train5 = train_generator5.n // train_generator5.batch_size

model1.fit_generator(generator=train_generator1, 
                     steps_per_epoch=step_size_train1,  
                     epochs=num_epochs)

model2.fit_generator(generator=train_generator2, 
                     steps_per_epoch=step_size_train2,  
                     epochs=num_epochs)

model3.fit_generator(generator=train_generator3, 
                     steps_per_epoch=step_size_train3, 
                     epochs=num_epochs)

"""model4.fit_generator(generator=train_generator4, 
                     steps_per_epoch=step_size_train4, 
                     epochs=num_epochs)

model5.fit_generator(generator=train_generator5, 
                     steps_per_epoch=step_size_train5, 
                     epochs=num_epochs)"""

Epoch 1/25
44/44 [==============================] - 17s 378ms/step - loss: 1.5830 - accuracy: 0.5572
Epoch 2/25
44/44 [==============================] - 11s 251ms/step - loss: 0.5749 - accuracy: 0.8181
Epoch 3/25
44/44 [==============================] - 11s 246ms/step - loss: 0.4114 - accuracy: 0.8537
Epoch 4/25
44/44 [==============================] - 11s 246ms/step - loss: 0.3585 - accuracy: 0.8788
Epoch 5/25
44/44 [==============================] - 11s 247ms/step - loss: 0.3009 - accuracy: 0.8976
Epoch 6/25
44/44 [==============================] - 11s 259ms/step - loss: 0.2730 - accuracy: 0.9056
Epoch 7/25
44/44 [==============================] - 11s 248ms/step - loss: 0.2159 - accuracy: 0.9252
Epoch 8/25
44/44 [==============================] - 11s 246ms/step - loss: 0.1811 - accuracy: 0.9328
Epoch 9/25
44/44 [==============================] - 11s 245ms/step - loss: 0.1573 - accuracy: 0.9535
Epoch 10/25
44/44 [==============================] - 11s 246ms/step - loss: 0.1442 - accura

'model4.fit_generator(generator=train_generator4, \n                     steps_per_epoch=step_size_train4, \n                     epochs=num_epochs)\n\nmodel5.fit_generator(generator=train_generator5, \n                     steps_per_epoch=step_size_train5, \n                     epochs=num_epochs)'

In [13]:
import cv2

test_data_name = []
test_data = []

for dirname, _, filenames in os.walk('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test'):
    for filename in filenames:
        test_data_name.append(filename)
        image = cv2.imread(os.path.join(dirname, filename))
        test_data.append(image)
        
print("Number of test data =", len(test_data))

Number of test data = 1040


In [14]:
def processing(image_data):
    height = 256
    width = 256
    dim = (width, height)
    res_img = []
    
    for i in range(len(image_data)):
        res = cv2.resize(image_data[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    
    return res_img

test_data_reshape = processing(test_data)

print(test_data_reshape[0].shape)

(256, 256, 3)


In [15]:
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = preprocess_input(test_data_reshape[i])
    
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = test_data_reshape[i].reshape(-1, 256, 256, 3)

In [16]:
print("id,label")

#maybe_wrong = []

for i in range(len(test_data_reshape)):
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    model1_pred = model1.predict(test_data_reshape[i])
    model2_pred = model2.predict(test_data_reshape[i])
    model3_pred = model3.predict(test_data_reshape[i])
    #model4_pred = model4.predict(test_data_reshape[i])
    #model5_pred = model5.predict(test_data_reshape[i])
    all_model_pred = [model1_pred[i] + model2_pred[i] + model3_pred[i] for i in range(len(model1_pred))] 
    print(name + "," + label_map[np.argmax(all_model_pred)])
# + model4_pred[i] + model5_pred[i]

id,label
image_0545,bedroom
image_0565,tallbuilding
image_0897,opencountry
image_0007,forest
image_0438,street
image_0741,livingroom
image_1037,coast
image_0706,opencountry
image_0698,street
image_0928,mountain
image_0270,kitchen
image_0472,highway
image_0019,insidecity
image_0442,livingroom
image_0247,mountain
image_0673,suburb
image_0984,mountain
image_0208,suburb
image_0364,forest
image_0508,mountain
image_1026,suburb
image_0463,forest
image_0826,suburb
image_0138,insidecity
image_0268,mountain
image_0832,suburb
image_0248,suburb
image_0885,suburb
image_0140,bedroom
image_0622,insidecity
image_0639,street
image_0117,office
image_0910,office
image_0127,street
image_0621,insidecity
image_0607,kitchen
image_0401,kitchen
image_0099,bedroom
image_0682,kitchen
image_0665,coast
image_0733,insidecity
image_0229,suburb
image_0726,forest
image_0045,street
image_0232,forest
image_0869,forest
image_0415,street
image_0167,bedroom
image_0223,street
image_1009,tallbuilding
image_0372,livingroom
im

image_0710,tallbuilding
image_0958,livingroom
image_0236,highway
image_0709,highway
image_0755,bedroom
image_0227,bedroom
image_1004,insidecity
image_1005,kitchen
image_0246,coast
image_0873,mountain
image_0388,opencountry
image_0421,kitchen
image_0283,kitchen
image_0827,forest
image_0875,coast
image_0536,bedroom
image_0393,tallbuilding
image_0404,kitchen
image_0802,tallbuilding
image_0195,insidecity
image_0109,office
image_0080,mountain
image_0196,livingroom
image_0115,street
image_1007,highway
image_0305,suburb
image_0782,street
image_0164,coast
image_0940,street
image_0325,tallbuilding
image_0539,highway
image_0573,insidecity
image_0448,coast
image_0801,mountain
image_0661,office
image_0969,suburb
image_0583,bedroom
image_0234,street
image_0943,bedroom
image_0563,highway
image_0695,coast
image_1027,mountain
image_0606,insidecity
image_0328,bedroom
image_0558,street
image_0572,tallbuilding
image_1032,insidecity
image_0589,coast
image_0748,opencountry
image_0900,bedroom
image_0071,hig

image_0090,suburb
image_0252,suburb
image_0713,mountain
image_0121,insidecity
image_0387,kitchen
image_0886,office
image_0483,highway
image_0822,mountain
image_0686,livingroom
image_0689,street
image_0031,forest
image_0990,tallbuilding
image_0211,kitchen
image_0852,forest
image_0313,forest
image_0815,insidecity
image_0011,livingroom
image_0664,forest
image_0039,suburb
image_0935,opencountry
image_0737,insidecity
image_0848,kitchen
image_0424,coast
image_0514,tallbuilding
image_0125,coast
image_0383,bedroom
image_0452,highway
image_0355,tallbuilding
image_0435,coast
image_0315,suburb
image_0101,coast
image_0546,mountain
image_0254,livingroom
image_0224,opencountry
image_0756,insidecity
image_0430,forest
image_0296,tallbuilding
image_0633,opencountry
image_1002,forest
image_0879,opencountry
image_0179,insidecity
image_0553,tallbuilding
image_0915,forest
image_0136,tallbuilding
image_0385,highway
image_0029,forest
image_0060,forest
image_0672,insidecity
image_0976,suburb
image_0076,inside

In [17]:
voting = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
#print(len(voting))

print("id,label")

for i in range(len(test_data_reshape)):
    voting = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    model1_pred = model1.predict(test_data_reshape[i])
    model2_pred = model2.predict(test_data_reshape[i])
    model3_pred = model3.predict(test_data_reshape[i])
    #model4_pred = model4.predict(test_data_reshape[i])
    #model5_pred = model5.predict(test_data_reshape[i])

    voting[np.argmax(model1_pred)] += 1
    voting[np.argmax(model2_pred)] += 1
    voting[np.argmax(model3_pred)] += 1
    #voting[np.argmax(model4_pred)] += 1
    #voting[np.argmax(model5_pred)] += 1
    print(name + "," + label_map[np.argmax(voting)])

id,label
image_0545,bedroom
image_0565,tallbuilding
image_0897,opencountry
image_0007,forest
image_0438,street
image_0741,livingroom
image_1037,coast
image_0706,opencountry
image_0698,street
image_0928,mountain
image_0270,kitchen
image_0472,highway
image_0019,insidecity
image_0442,livingroom
image_0247,mountain
image_0673,suburb
image_0984,mountain
image_0208,suburb
image_0364,forest
image_0508,mountain
image_1026,suburb
image_0463,forest
image_0826,suburb
image_0138,insidecity
image_0268,mountain
image_0832,suburb
image_0248,suburb
image_0885,suburb
image_0140,bedroom
image_0622,insidecity
image_0639,street
image_0117,office
image_0910,office
image_0127,street
image_0621,insidecity
image_0607,kitchen
image_0401,kitchen
image_0099,bedroom
image_0682,kitchen
image_0665,coast
image_0733,insidecity
image_0229,suburb
image_0726,forest
image_0045,street
image_0232,forest
image_0869,forest
image_0415,street
image_0167,bedroom
image_0223,street
image_1009,tallbuilding
image_0372,livingroom
im

image_0236,highway
image_0709,highway
image_0755,bedroom
image_0227,bedroom
image_1004,insidecity
image_1005,kitchen
image_0246,coast
image_0873,mountain
image_0388,opencountry
image_0421,kitchen
image_0283,kitchen
image_0827,forest
image_0875,coast
image_0536,bedroom
image_0393,tallbuilding
image_0404,kitchen
image_0802,tallbuilding
image_0195,insidecity
image_0109,office
image_0080,mountain
image_0196,livingroom
image_0115,street
image_1007,highway
image_0305,suburb
image_0782,street
image_0164,coast
image_0940,street
image_0325,tallbuilding
image_0539,highway
image_0573,insidecity
image_0448,coast
image_0801,mountain
image_0661,office
image_0969,suburb
image_0583,bedroom
image_0234,street
image_0943,bedroom
image_0563,highway
image_0695,coast
image_1027,mountain
image_0606,insidecity
image_0328,bedroom
image_0558,street
image_0572,tallbuilding
image_1032,insidecity
image_0589,coast
image_0748,opencountry
image_0900,bedroom
image_0071,highway
image_0191,mountain
image_0425,livingroom

image_0252,suburb
image_0713,mountain
image_0121,insidecity
image_0387,kitchen
image_0886,office
image_0483,highway
image_0822,mountain
image_0686,livingroom
image_0689,street
image_0031,forest
image_0990,tallbuilding
image_0211,kitchen
image_0852,forest
image_0313,forest
image_0815,insidecity
image_0011,livingroom
image_0664,forest
image_0039,suburb
image_0935,opencountry
image_0737,insidecity
image_0848,kitchen
image_0424,coast
image_0514,tallbuilding
image_0125,coast
image_0383,bedroom
image_0452,highway
image_0355,tallbuilding
image_0435,coast
image_0315,suburb
image_0101,coast
image_0546,mountain
image_0254,livingroom
image_0224,opencountry
image_0756,insidecity
image_0430,forest
image_0296,tallbuilding
image_0633,opencountry
image_1002,forest
image_0879,opencountry
image_0179,insidecity
image_0553,tallbuilding
image_0915,forest
image_0136,tallbuilding
image_0385,highway
image_0029,forest
image_0060,forest
image_0672,insidecity
image_0976,suburb
image_0076,insidecity
image_0818,ta